In [ ]:
import plotly.graph_objects as go

# --- 1. CONFIGURAZIONE E HELPER ---
TOTAL_POPULATION = 100

# Funzione per convertire HEX in RGBA (risolve il tuo errore)
def hex_to_rgba(hex_color, alpha=0.4):
    hex_color = hex_color.lstrip('#')
    if len(hex_color) == 6:
        r = int(hex_color[0:2], 16)
        g = int(hex_color[2:4], 16)
        b = int(hex_color[4:6], 16)
        return f'rgba({r},{g},{b},{alpha})'
    return hex_color

# Palette Colori (Standardizzate)
COLORS = {
    "TRAD": "#4D4D4D",  # Grigio Scuro
    "HTP": "#2E86C1",   # Blu
    "ECIG": "#E67E22",  # Arancione
    "START": "#17202A"  # Nero/Grigio molto scuro
}

# --- 2. DEFINIZIONE NODI ---
# Indici definiti manualmente per garantire l'ordine
# 0: Start
# 1-3: Livello 1 (Primo Prodotto)
# 4-12: Livello 2 (Intermedi)
# 13-18: Livello 3 (DESTINAZIONI FINALI - Le 6 Chiavi richieste)

labels = [
    "Totale Fumatori",                                      # 0
    "Prova TRAD (1°)", "Prova HTP (1°)", "Prova ECIG (1°)", # 1, 2, 3
    # Intermedi (servono per instradare il flusso ma non sono terminali)
    "Fumo TRAD (int)", "Prova HTP (int)", "Prova ECIG (int)", # 4, 5, 6
    "Prova TRAD (int)", "Fumo HTP (int)", "Prova ECIG (int)", # 7, 8, 9
    "Prova TRAD (int)", "Prova HTP (int)", "Fumo ECIG (int)", # 10, 11, 12
    # LE 6 CHIAVI FINALI
    "FUMO TRAD", "FUMO HTP", "FUMO ECIG",                   # 13, 14, 15
    "PROVA TRAD", "PROVA HTP", "PROVA ECIG"                 # 16, 17, 18
]

# Assegnazione Colori ai Nodi
node_colors = [
    COLORS["START"],
    COLORS["TRAD"], COLORS["HTP"], COLORS["ECIG"],
    COLORS["TRAD"], COLORS["HTP"], COLORS["ECIG"],
    COLORS["TRAD"], COLORS["HTP"], COLORS["ECIG"],
    COLORS["TRAD"], COLORS["HTP"], COLORS["ECIG"],
    COLORS["TRAD"], COLORS["HTP"], COLORS["ECIG"], # Finali Fumo
    COLORS["TRAD"], COLORS["HTP"], COLORS["ECIG"]  # Finali Prova
]

source = []
target = []
value = []
link_colors = []

# Helper per aggiungere i flussi in modo pulito
def add_flow(src, tgt, val, color_key):
    source.append(src)
    target.append(tgt)
    value.append(val)
    # Usa la funzione hex_to_rgba per evitare l'errore ValueError
    link_colors.append(hex_to_rgba(COLORS[color_key], 0.4))

# --- 3. CALCOLO DEI FLUSSI ---

# LIVELLO 1: Da Totale a Primo Tentativo
v_L1_Trad = TOTAL_POPULATION * 0.594
v_L1_Htp = TOTAL_POPULATION * 0.055
v_L1_Ecig = TOTAL_POPULATION * 0.351

add_flow(0, 1, v_L1_Trad, "TRAD")
add_flow(0, 2, v_L1_Htp, "HTP")
add_flow(0, 3, v_L1_Ecig, "ECIG")

# LIVELLO 2 & 3: Sviluppo dei Rami

# --- Ramo TRAD (Nodo 1) ---
v_1_FumoTrad = v_L1_Trad * 0.390
v_1_ProvaHtp = v_L1_Trad * 0.191
v_1_ProvaEcig = v_L1_Trad * 0.419

add_flow(1, 4, v_1_FumoTrad, "TRAD")
add_flow(1, 5, v_1_ProvaHtp, "HTP")
add_flow(1, 6, v_1_ProvaEcig, "ECIG")

# Dal nodo intermedio 4 (Fumo TRAD) -> Finali
add_flow(4, 17, v_1_FumoTrad * 0.423, "HTP")  # -> PROVA HTP
add_flow(4, 18, v_1_FumoTrad * 0.523, "ECIG") # -> PROVA ECIG
# (Nota: STOP 5.4% omesso per richiesta "solo 6 chiavi")

# Dal nodo intermedio 5 (Prova HTP) -> Finali
add_flow(5, 13, v_1_ProvaHtp * 0.175, "TRAD") # -> FUMO TRAD
add_flow(5, 14, v_1_ProvaHtp * 0.156, "HTP")  # -> FUMO HTP
add_flow(5, 18, v_1_ProvaHtp * 0.669, "ECIG") # -> PROVA ECIG

# Dal nodo intermedio 6 (Prova ECIG) -> Finali
add_flow(6, 13, v_1_ProvaEcig * 0.192, "TRAD")
add_flow(6, 17, v_1_ProvaEcig * 0.496, "HTP")
add_flow(6, 15, v_1_ProvaEcig * 0.311, "ECIG")


# --- Ramo HTP (Nodo 2) ---
v_2_ProvaTrad = v_L1_Htp * 0.483
v_2_FumoHtp = v_L1_Htp * 0.232
v_2_ProvaEcig = v_L1_Htp * 0.285

add_flow(2, 7, v_2_ProvaTrad, "TRAD")
add_flow(2, 8, v_2_FumoHtp, "HTP")
add_flow(2, 9, v_2_ProvaEcig, "ECIG")

# Dai nodi intermedi -> Finali
add_flow(7, 13, v_2_ProvaTrad * 0.267, "TRAD")
add_flow(7, 14, v_2_ProvaTrad * 0.267, "HTP")
add_flow(7, 18, v_2_ProvaTrad * 0.466, "ECIG")

add_flow(8, 16, v_2_FumoHtp * 0.524, "TRAD")
add_flow(8, 18, v_2_FumoHtp * 0.476, "ECIG")

add_flow(9, 16, v_2_ProvaEcig * 0.650, "TRAD")
add_flow(9, 14, v_2_ProvaEcig * 0.201, "HTP")
add_flow(9, 15, v_2_ProvaEcig * 0.148, "ECIG")


# --- Ramo ECIG (Nodo 3) ---
v_3_ProvaTrad = v_L1_Ecig * 0.467
v_3_ProvaHtp = v_L1_Ecig * 0.260
v_3_FumoEcig = v_L1_Ecig * 0.273

add_flow(3, 10, v_3_ProvaTrad, "TRAD")
add_flow(3, 11, v_3_ProvaHtp, "HTP")
add_flow(3, 12, v_3_FumoEcig, "ECIG")

# Dai nodi intermedi -> Finali
add_flow(10, 13, v_3_ProvaTrad * 0.174, "TRAD")
add_flow(10, 17, v_3_ProvaTrad * 0.602, "HTP")
add_flow(10, 15, v_3_ProvaTrad * 0.224, "ECIG")

add_flow(11, 16, v_3_ProvaHtp * 0.842, "TRAD")
add_flow(11, 14, v_3_ProvaHtp * 0.050, "HTP")
add_flow(11, 15, v_3_ProvaHtp * 0.107, "ECIG")

add_flow(12, 16, v_3_FumoEcig * 0.684, "TRAD")
add_flow(12, 17, v_3_FumoEcig * 0.316, "HTP")


# --- 4. VISUALIZZAZIONE ---
fig = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    valuesuffix = " Utenti",
    node=dict(
        pad=30,
        thickness=20,
        line=dict(color="white", width=1),
        label=labels,
        color=node_colors
    ),
    link=dict(
        source=source,
        target=target,
        value=value,
        color=link_colors
    )
)])

fig.update_layout(
    title_text="<b>Storia del consumo dei fumatori di sigaretta tradizionale(ultimi 30 giorni rispetto al sondaggio)</b><br><span style='font-size:12px;color:grey'>fonte: Rapporto Nazionale sul Tabagismo 2024</span>",
    font=dict(size=14, color="#2c3e50", family="Arial"),
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=800,
)

fig.show()